In [1]:
import torch
#clear memory better
with torch.no_grad():
    torch.cuda.empty_cache()
import numpy as np
import util.npose_util as nu
import os
import pathlib
import dgl
from dgl import backend as F
import torch_geometric
from torch.utils.data import random_split, DataLoader, Dataset
from typing import Dict
from torch import Tensor
from dgl import DGLGraph
from torch import nn
# from chemical import cos_ideal_NCAC #from RoseTTAFold2
from torch import einsum
import time
torch.cuda.is_available()

True

In [2]:
# from data_rigid_diffuser import so3_diffuser
# from data_rigid_diffuser import r3_diffuser
# from scipy.spatial.transform import Rotation
# from data_rigid_diffuser import rigid_utils as ru
# import yaml
from data_rigid_diffuser.diffuser import FrameDiffNoise
from gudiff_model import Data_Graph
from gudiff_model.Data_Graph import build_npose_from_coords, dump_coord_pdb, define_graph_edges, make_pe_encoding
from gudiff_model.Data_Graph import Helix4_Dataset, Make_KNN_MP_Graphs

In [3]:
from se3_transformer.model.basis import get_basis, update_basis_with_fused
from se3_transformer.model.transformer import Sequential, SE3Transformer
from se3_transformer.model.transformer_topk import SE3Transformer_topK
from se3_transformer.model.FAPE_Loss import FAPE_loss, Qs2Rs, normQ
from se3_transformer.model.layers.attentiontopK import AttentionBlockSE3
from se3_transformer.model.layers.linear import LinearSE3
from se3_transformer.model.layers.convolution import ConvSE3, ConvSE3FuseLevel
from se3_transformer.model.layers.norm import NormSE3
from se3_transformer.model.layers.pooling import GPooling, Latent_Unpool, Unpool_Layer
from se3_transformer.runtime.utils import str2bool, to_cuda
from se3_transformer.model.fiber import Fiber
from se3_transformer.model.transformer import get_populated_edge_features

In [4]:
#indices for, unsure if needed
CA = Data_Graph.CA
N = Data_Graph.N
C = Data_Graph.C

#find better way to incorporate coord_scale

#needed
N_CA_dist = (Data_Graph.N_CA_dist/10.).to('cuda')
C_CA_dist = (Data_Graph.C_CA_dist/10.).to('cuda')



In [5]:
# data_path_str  = 'data/h4_ca_coords.npz'
# test_limit = 1028
# rr = np.load(data_path_str)
# ca_coords = [rr[f] for f in rr.files][0][:test_limit,:,:3]
# ca_coords.shape

# getting N-Ca, Ca-C vectors to add as typeI features
#apa = apart helices for val/train split
#tog = together helices for val/train split
apa_path_str  = 'data_npose/h4_apa_coords.npz'
tog_path_str  = 'data_npose/h4_tog_coords.npz'

#grab the first 3 atoms which are N,CA,C
test_limit = 5048
rr = np.load(apa_path_str)
coords_apa = [rr[f] for f in rr.files][0][:test_limit,:]

rr = np.load(tog_path_str)
coords_tog = [rr[f] for f in rr.files][0][:test_limit,:]

In [6]:
from gudiff_model.Graph_UNet import GraphUNet

In [11]:
def model_step(backbone_dict, noised_dict, batched_t, scores_scales, graph_maker, graph_unet, train=True):
    
    CA_t  = backbone_dict['CA'].reshape(B, L, 3).to('cuda')
    NC_t = CA_t + backbone_dict['N_CA'].reshape(B, L, 3).to('cuda')
    CC_t = CA_t + backbone_dict['C_CA'].reshape(B, L, 3).to('cuda')
    true =  torch.cat((NC_t,CA_t,CC_t),dim=2).reshape(B,L,3,3)
    
    CA_n  = noised_dict['CA'].reshape(B, L, 3).to('cuda')
    NC_n = CA_n + noised_dict['N_CA'].reshape(B, L, 3).to('cuda')
    CC_n = CA_n + noised_dict['C_CA'].reshape(B, L, 3).to('cuda')
    noise_xyz =  torch.cat((NC_n,CA_n,CC_n),dim=2).reshape(B,L,3,3)
    
    x = graph_maker.prep_for_network(noised_dict)
    out = graph_unet(x, batched_t)
    CA_p = out['1'][:,0,:].reshape(B, L, 3)+CA_n #translation of Calpha
    Qs = out['1'][:,1,:] # rotation
    Qs = Qs.unsqueeze(1).repeat((1,2,1))
    Qs = torch.cat((torch.ones((B*L,2,1),device=Qs.device),Qs),dim=-1).reshape(B,L,2,4)
    Qs = normQ(Qs)
    Rs = Qs2Rs(Qs)
    N_C_to_Rot = torch.cat((noised_dict['N_CA'].reshape(B, L, 3).to('cuda'),
                            noised_dict['C_CA'].reshape(B, L, 3).to('cuda')),dim=2).reshape(B,L,2,1,3)

    
    
    rot_vecs = einsum('bnkij,bnkhj->bnki',Rs, N_C_to_Rot)
    NC_p = CA_p + rot_vecs[:,:,0,:].to('cuda')*N_CA_dist
    CC_p = CA_p + rot_vecs[:,:,1,:].reshape(B, L, 3).to('cuda')*C_CA_dist

    pred = torch.cat((NC_p,CA_p,CC_p),dim=2).reshape(B,L,3,3)
    
    tloss, loss = FAPE_loss(pred.unsqueeze(0), true, scores_scales)
    
    return tloss

In [12]:
def get_noise_pred_true(backbone_dict, noised_dict, batched_t, graph_maker, graph_unet):
    
    CA_t  = backbone_dict['CA'].reshape(B, L, 3).to('cuda')
    NC_t = CA_t + backbone_dict['N_CA'].reshape(B, L, 3).to('cuda')*N_CA_dist
    CC_t = CA_t + backbone_dict['C_CA'].reshape(B, L, 3).to('cuda')*C_CA_dist
    true =  torch.cat((NC_t,CA_t,CC_t),dim=2).reshape(B,L,3,3)
    
    CA_n  = noised_dict['CA'].reshape(B, L, 3).to('cuda')
    NC_n = CA_n + noised_dict['N_CA'].reshape(B, L, 3).to('cuda')*N_CA_dist
    CC_n = CA_n + noised_dict['C_CA'].reshape(B, L, 3).to('cuda')*C_CA_dist
    noise_xyz =  torch.cat((NC_n,CA_n,CC_n),dim=2).reshape(B,L,3,3)
    
    x = graph_maker.prep_for_network(noised_dict)
    out = graph_unet(x, batched_t)
    CA_p = out['1'][:,0,:].reshape(B, L, 3)+CA_n #translation of Calpha
    Qs = out['1'][:,1,:] # rotation
    Qs = Qs.unsqueeze(1).repeat((1,2,1))
    Qs = torch.cat((torch.ones((B*L,2,1),device=Qs.device),Qs),dim=-1).reshape(B,L,2,4)
    Qs = normQ(Qs)
    Rs = Qs2Rs(Qs)
    N_C_to_Rot = torch.cat((noised_dict['N_CA'].reshape(B, L, 3).to('cuda'),
                            noised_dict['C_CA'].reshape(B, L, 3).to('cuda')),dim=2).reshape(B,L,2,1,3)
    
    
    rot_vecs = einsum('bnkij,bnkhj->bnki',Rs, N_C_to_Rot)
    NC_p = CA_p + rot_vecs[:,:,0,:].to('cuda')*N_CA_dist
    CC_p = CA_p + rot_vecs[:,:,1,:].reshape(B, L, 3).to('cuda')*C_CA_dist

    pred = torch.cat((NC_p,CA_p,CC_p),dim=2).reshape(B,L,3,3)
    
    return true.to('cpu').numpy()*10, noise_xyz.to('cpu').numpy()*10, pred.detach().to('cpu').numpy()*10

def dump_tnp(true, noise, pred, t_val, e=0, numOut=1,outdir='output/'):
    
    if numOut>true.shape[0]:
        numOut = true.shape[0]
    
    for x in range(numOut):
        dump_coord_pdb(true[x], fileOut=f'{outdir}/true_{t_val[x]*100:.0f}_e{e}_{x}.pdb')
        dump_coord_pdb(noise[x], fileOut=f'{outdir}/noise_{t_val[x]*100:.0f}_e{e}_{x}.pdb')
        dump_coord_pdb(pred[x], fileOut=f'{outdir}/pred_{t_val[x]*100:.0f}_e{e}_{x}.pdb')
        
def visualize_model(bb_dict, noised_bb, batched_t, epoch, numOut=1, outdir='output/'):
    true, noise, pred = get_noise_pred_true(bb_dict, noised_bb, batched_t, gm, gu)
    dump_tnp(true,noise,pred, batched_t, e=epoch, numOut=numOut, outdir=f'{outdir}/models/')

In [13]:
def make_save_folder(name=''):
    base_folder = time.strftime(f'log/%y%b%d_%I%M%p_{name}/', time.localtime())
    if not os.path.exists(base_folder):
        os.makedirs(base_folder)
    subfolders = ['models']
    for subfolder in subfolders:
        if not os.path.exists(base_folder + subfolder):
            os.makedirs(base_folder + subfolder)
            
    return base_folder
        
def save_chkpt(model_path, model, optimizer, epoch, batch, val_losses, train_losses):
    """Save a training checkpoint
    Args:
        model_path (str): the path to save the model to
        model (nn.Module): the model to save
        optimizer (torch.optim.Optimizer): the optimizer to save
        epoch (int): the current epoch
        batch (int): the current batch in the epoch
        loss_domain (list of int): a list of the shared domain for val and training 
            losses
        val_losses (list of float): a list containing the validation losses
        train_losses (list of float): a list containing the training losses
    """
    os.makedirs(os.path.dirname(model_path), exist_ok=True)
    state_dict = dict()
    state_dict.update({'model':model.state_dict(),
                       'optimizer':optimizer.state_dict(),
                       'epoch':epoch,
                       'batch':batch,
                       'train_losses':train_losses,
                       'val_losses':val_losses
                       })
    torch.save(state_dict, f'{model_path}model_e{epoch}')
    
def load_model(model_path, model_class):
    """Load a saved model"""
    
    device = 'cuda:0'
    model = model_class()
    model.load_state_dict(torch.load(model_path)['model'])
    model.to(device)
    return model

In [14]:
B = 32
L=65
limit = 5048
h4_trainData = Helix4_Dataset(coords_tog[:limit])
h4_valData = Helix4_Dataset(coords_apa[:limit])
train_dL = DataLoader(h4_trainData, batch_size=B, shuffle=True, drop_last=True)
val_dL   = DataLoader(h4_valData, batch_size=B, shuffle=True, drop_last=True)


In [15]:
gu = GraphUNet(batch_size = B, num_layers_ca = 2).to('cuda')
#opti = torch.optim.Adam(gu.parameters(), lr=0.0005, weight_decay=5e-6)
gm = Make_KNN_MP_Graphs() #consider precalculating graphs for training
fdn= FrameDiffNoise()
useR3 = False




#visualize_T
vis_t = np.array([0.01,0.05,0.1,0.2,0.3,0.5,0.8,1.0])
vis_t = vis_t[None,...].repeat(int(np.ceil(B/len(vis_t))),axis=0).flatten()[:B]
#vis_t =  torch.tensor(vis_t, dtype=torch.float32).to('cuda')

In [16]:
#load model
model_path = '23Sep01_0242AM_full_diff_embed_T_btest'
modelFile = 'model_e489'
#opti = torch.optim.Adam(gu.parameters())
device = 'cuda:0'
gu.load_state_dict(torch.load(f'log/{model_path}/{modelFile}')['model'])
#opti.load_state_dict(torch.load(f'log/{model_path}/{modelFile}')['optimizer'])
#visualize_model(bb_dict, noised_bb, tv, 'test_start', numOut=8,outdir=f'output/')
#visualize_model(bb_dict, noised_bb, tv, 'test_start', numOut=8,outdir=f'log/{model_path}')
#save_chkpt(f'log/{model_path}/', gu, opti, 'test_start', B, avg_vloss, avg_tloss)

<All keys matched successfully>

In [17]:
test_iter = iter(train_dL)
test_batch = next(test_iter)

t=0.05
t_vec = np.ones((B,))*t
nd, tv, ss = fdn(test_batch, t_vec=t_vec, useR3=False)

tv = tv.to('cuda')
# train_loss = model_step(bb_dict, noised_bb, tv, gm, gu)

In [18]:
visualize_model(test_batch, nd, tv, 1000, numOut=1,outdir='output/')
#save_chkpt(model_path, gu, opti, 1000, B, 0, 0)

/home/nwoodall/miniconda3/envs/se33/lib/python3.9/site-packages/dgl/backend/pytorch/tensor.py:449: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), (


In [25]:
#t=0.1
#t_vec = np.ones((B,))*t
#print(t_vec)
#model_path = make_save_folder(name=f'full_diff_embed_T_btest')
num_epochs = 100
e_start= 190
save_per=10
avg_vloss=0

for e in range(e_start, e_start+num_epochs):
    
    running_tloss = 0 
    start = time.time()
    for i, bb_dict in enumerate(train_dL):
        noised_bb, tv, ss = fdn(bb_dict,t_vec=None, useR3=useR3)
        tv = tv.to('cuda')
        ss = ss.to('cuda')
        train_loss = model_step(bb_dict, noised_bb, tv, ss, gm, gu)
        opti.zero_grad()
        train_loss.backward()
        opti.step()

        running_tloss += train_loss.detach().cpu()
    
    end = time.time()
    avg_tloss = running_tloss/(i+1)
    print(f'Average Train Loss Epoch {e}: {avg_tloss};   Epoch time: {end-start:.0f}')

    if e %save_per==save_per-1:
        with torch.no_grad():
            running_vloss = 0
            for i, bb_dictv in enumerate(val_dL):
                noised_bb, tv, ss = fdn(bb_dictv,t_vec=None, useR3=useR3)
                tv = tv.to('cuda')
                ss = ss.to('cuda')
                valid_loss = model_step(bb_dictv, noised_bb, tv, ss, gm, gu)
                running_vloss += valid_loss
                
        avg_vloss = running_vloss/(i+1)
        print(f'Average Valid Loss Epoch {e}: {avg_vloss}')
                
        noised_bb, tv, ss = fdn(bb_dict, t_vec=vis_t)
        tv = tv.to('cuda')
        visualize_model(bb_dict, noised_bb, tv, e, numOut=8,outdir=f'log/{model_path}')
        save_chkpt(f'log/{model_path}', gu, opti, e, B, avg_vloss, avg_tloss)
#         visualize_model(bb_dict, noised_bb, tv, e, numOut=8,outdir=f'{model_path}')
#         save_chkpt(f'{model_path}', gu, opti, e, B, avg_vloss, avg_tloss)


KeyboardInterrupt: 

In [39]:
test_iter = iter(train_dL)
test_batch = next(test_iter)

t=0.05
t_vec = np.ones((B,))*t
nd, tv, ss = fdn(test_batch, t_vec=t_vec, useR3=False)

tv = tv.to('cuda')
# train_loss = model_step(bb_dict, noised_bb, tv, gm, gu)

In [40]:
visualize_model(test_batch, nd, tv, 1000, numOut=8,outdir='output/')
#save_chkpt('output/', gu, opti, 1000, B, 0, 0)

In [81]:
#From
#https://math.stackexchange.com/questions/939229/unit-quaternion-to-a-scalar-power

#test code
# Q = torch.tensor([0,0,1,0],dtype=torch.float)[None,None,None,...].repeat(2,3,2,1) #180 rotation about Y axis
# rv = torch.tensor([[0,0,1]],dtype=torch.float)[None,None,...].repeat(2,3,2,1) #unit Z
# Q = normQ(Q)
#qp = powerQ(Q,0.5) rotate halfway (aka x-axis)
#rv = rv.reshape((2,3,2,1,3))
#rot_vecs = einsum('bnkij,bnkhj->bnki',Rs, rv) #should be x-axis

def powerQ(quat_in, power):
    """Quaternion to the power, represent number of times to rotate Q. Only works on unit Q"""
    return expQ(scaleQ(lnQ(quat_in),power))


def expQ(quat_in, eps=1e-9):
    quat_out = torch.zeros_like(quat_in, device=quat_in.device)
    r = torch.sqrt(torch.sum(torch.square(quat_in[...,1:]),axis=-1,keepdim=True)+eps)
    et = torch.exp(quat_in[...,0][...,None])
    s = et*torch.sin(r)/r
    s[torch.where(r<eps)[0]] = 0
    
    quat_out[...,0][...,None] = et*torch.cos(r)
    quat_out[...,1] = s[...,0]*quat_in[...,1]
    quat_out[...,2] = s[...,0]*quat_in[...,2]
    quat_out[...,3] = s[...,0]*quat_in[...,3]
    
    return quat_out

#fix in other location w calc
def lnQ(quat_in, eps=1e-9):
    quat_out = torch.zeros_like(quat_in, device=quat_in.device)
    r = torch.sqrt(torch.sum(torch.square(quat_in[...,1:]),axis=-1,keepdim=True)+eps)
    t = torch.atan2(r,quat_in[...,0][...,None])/r
    t[torch.where(r<eps)[0]] = 0
        
    quat_out[...,0][...,None] = 0.5*torch.log(torch.sum(torch.square(quat_in),axis=-1,keepdim=True)+eps)
    quat_out[...,1] = t[...,0]*quat_in[...,1]
    quat_out[...,2] = t[...,0]*quat_in[...,2]
    quat_out[...,3] = t[...,0]*quat_in[...,3]
    
    return quat_out
    

def scaleQ(quat_in, scale):
    return torch.multiply(quat_in,scale)

In [ ]:
def model_step(backbone_dict, noised_dict, batched_t, scores_scales, graph_maker, graph_unet, train=True):
    
    CA_t  = backbone_dict['CA'].reshape(B, L, 3).to('cuda')
    NC_t = CA_t + backbone_dict['N_CA'].reshape(B, L, 3).to('cuda')
    CC_t = CA_t + backbone_dict['C_CA'].reshape(B, L, 3).to('cuda')
    true =  torch.cat((NC_t,CA_t,CC_t),dim=2).reshape(B,L,3,3)
    
    CA_n  = noised_dict['CA'].reshape(B, L, 3).to('cuda')
    NC_n = CA_n + noised_dict['N_CA'].reshape(B, L, 3).to('cuda')
    CC_n = CA_n + noised_dict['C_CA'].reshape(B, L, 3).to('cuda')
    noise_xyz =  torch.cat((NC_n,CA_n,CC_n),dim=2).reshape(B,L,3,3)
    
    x = graph_maker.prep_for_network(noised_dict)
    out = graph_unet(x, batched_t)
    CA_p = out['1'][:,0,:].reshape(B, L, 3)+CA_n #translation of Calpha
    Qs = out['1'][:,1,:] # rotation
    Qs = Qs.unsqueeze(1).repeat((1,2,1))
    Qs = torch.cat((torch.ones((B*L,2,1),device=Qs.device),Qs),dim=-1).reshape(B,L,2,4)
    Qs = normQ(Qs)
    Rs = Qs2Rs(Qs)
    N_C_to_Rot = torch.cat((noised_dict['N_CA'].reshape(B, L, 3).to('cuda'),
                            noised_dict['C_CA'].reshape(B, L, 3).to('cuda')),dim=2).reshape(B,L,2,1,3)

    
    
    rot_vecs = einsum('bnkij,bnkhj->bnki',Rs, N_C_to_Rot)
    NC_p = CA_p + rot_vecs[:,:,0,:].to('cuda')*N_CA_dist
    CC_p = CA_p + rot_vecs[:,:,1,:].reshape(B, L, 3).to('cuda')*C_CA_dist

    pred = torch.cat((NC_p,CA_p,CC_p),dim=2).reshape(B,L,3,3)
    
    tloss, loss = FAPE_loss(pred.unsqueeze(0), true, scores_scales)
    
    return tloss

In [19]:
def noise_test(backbone_dict, noised_dict):
    CA_t  = bb_dict['CA'].reshape(B, L, 3).to('cuda')
    NC_t = CA_t + bb_dict['N_CA'].reshape(B, L, 3).to('cuda')*N_CA_dist
    CC_t = CA_t + bb_dict['C_CA'].reshape(B, L, 3).to('cuda')*C_CA_dist
    true =  torch.cat((NC_t,CA_t,CC_t),dim=2).reshape(B,L,3,3)
    
    CA_n  = noised_dict['CA'].reshape(B, L, 3).to('cuda')
    NC_n = CA_n + noised_dict['N_CA'].reshape(B, L, 3).to('cuda')*N_CA_dist
    CC_n = CA_n + noised_dict['C_CA'].reshape(B, L, 3).to('cuda')*C_CA_dist
    noise_xyz =  torch.cat((NC_n,CA_n,CC_n),dim=2).reshape(B,L,3,3)
    return true.to('cpu').numpy()*10, noise_xyz.to('cpu').numpy()*10

In [20]:
from util.npose_util import makePointPDB
#gds = Graph_RadiusMP_4H_Dataset(coords_tog[:100], 10, mp_stride = 3)
def view_mp_graph(mps: DGLGraph, coords: np.array ):
    p = mps.ndata['pos']*10
    
    to = np.concatenate((coords, np.ones_like(coords)[:,:,0][...,None]),axis=2)
    
    makePointPDB(p,'test.pdb',outDirec='output')
    nu.dump_npdb(to,'output/test2.pdb')
#view_mp_graph(gds.mpSelfGraphList[0], coords_tog[0])